In [2]:
#!pip install llama-index ollama

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 111.4 kB/s  0:01:55m0:00:0200:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 193.3 kB/s  0:00:05 eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 181.6 kB/s  0:01:20m0:00:0200:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 146.4 kB/s  0:00:14 eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 456.8 kB/s  0:00:03 eta 0:00:

In [5]:
#!pip install llmsherpa

In [13]:
#!python3 query_tables.py

According to the table, Google's operating margin for Q1 2024 was 32%. This is indicated in the "Operating income" and "Operating margin" rows of the table.
To find the percentage of Net income is of the Revenues, you can divide the Net Income by the Revenues and then multiply by 100.

For Q1 2024:

Net Income = $23,662
Revenues = $80,539

Percentage = ($23,662 ÷ $80,539) x 100 = 29.5%

Therefore, the percentage of Net income is approximately 29.5% of the Revenues for Q1 2024.


In [ ]:
!!pip install sentence-transformers

In [ ]:
from llmsherpa.readers import LayoutPDFReader
from llama_index.llms.ollama import Ollama

from llama_index.core import VectorStoreIndex
from llama_index.core import Document, ServiceContext, Settings
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding# no need
from llama_index.core import Settings


# Initialize LLm
llm = Ollama(model="llama3", request_timeout=60.0)

llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all"
pdf_url = "https://s206.q4cdn.com/479360582/files/doc_financials/2024/q1/2024q1-alphabet-earnings-release-pdf.pdf"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

# Read PDF
doc = pdf_reader.read_pdf(pdf_url)

In [15]:
# Get data from the Section by Title
selected_section = None
for section in doc.sections():
    if 'Q1 2024 Financial Highlights' in section.title:
        selected_section = section
        break

# Convert the output in HTML format
context = selected_section.to_html(include_children=True, recurse=True)
question = "What was Google's operating margin for 2024"
resp = llm.complete(
    f"read this table and answer question: {question}:\n{context}")
print(resp.text)

question = "What % Net income is of the Revenues?"
resp = llm.complete(
    f"read this table and answer question: {question}:\n{context}")
print(resp.text)

According to the table, Google's operating margin for 2024 was 32%. This is based on the "Operating income" and "Revenues" figures provided in the table.
To calculate the percentage of Net income to Revenues, we can use the figures from Q1 2024:

Revenues: $80,539
Net Income: $23,662

Percentage = (Net Income / Revenues) x 100%

Percentage = ($23,662 / $80,539) x 100% ≈ 29.5%

So, Net income is approximately 29.5% of the Revenues.


In [16]:
def get_all_sections_with_tables(doc):
    sections_with_tables = []
    for section in doc.sections():
        html = section.to_html(include_children=True, recurse=True)
        if '<table' in html.lower():
            sections_with_tables.append({
                'title': section.title,
                'html': html
            })
            print(f"Found table in section: {section.title}")
    return sections_with_tables


def query_all_tables(question: str, sections_with_tables: list) -> str:
    combined_context = "\n\n".join([
        f"=== Section: {s['title']} ===\n{s['html']}" 
        for s in sections_with_tables
    ])
    
    prompt = f"""Based on the following tables from a financial report, answer this question:

Question: {question}

Tables:
{combined_context}

Please provide a clear answer. If calculation is needed, show your steps."""
    
    resp = llm.complete(prompt)
    return resp.text


# ========== Task b:  ==========

def test_reasoning_capabilities(sections_with_tables: list):
    test_cases = [
        {
            "question": "What was Google's operating margin for Q1 2024?",
            "type": "lookup",
            "expected": "应该返回32%"
        },
        
        {
            "question": "What is the sum of Revenues for Q1 2023 and Q1 2024?",
            "type": "addition",
            "expected": "应该是 69,787 + 80,539 = 150,326"
        },
        
        {
            "question": "How much did Operating income increase from Q1 2023 to Q1 2024?",
            "type": "subtraction",
            "expected": "25,472 - 17,415 = 8,057"
        },
        
        {
            "question": "What percentage is Net income of Revenues for Q1 2024?",
            "type": "percentage",
            "expected": "23,662 / 80,539 ≈ 29.4%"
        },
        
        {
            "question": "Which metric showed the largest percentage increase from 2023 to 2024?",
            "type": "comparison",
            "expected": "需要比较多个指标的增长率"
        },
        
        {
            "question": "If the revenue growth rate from Q1 2023 to Q1 2024 continues, what would be the projected Q1 2025 revenue?",
            "type": "projection",
            "expected": "需要计算增长率(15%)然后应用"
        },
    ]
    
    results = []
    for test in test_cases:
        print(f"\n{'='*60}")
        print(f"[{test['type'].upper()}] {test['question']}")
        print(f"Expected behavior: {test['expected']}")
        
        answer = query_all_tables(test['question'], sections_with_tables)
        print(f"\nLLM Answer: {answer}")
        
        results.append({
            'type': test['type'],
            'question': test['question'],
            'answer': answer,
            'expected': test['expected']
        })
    
    return results


if __name__ == "__main__":
    # Task a
    print("=" * 60)
    print("Task a: Loading all sections with tables...")
    print("=" * 60)
    sections = get_all_sections_with_tables(doc)
    print(f"\nFound {len(sections)} sections with tables")
    
    # Task b
    print("\n" + "=" * 60)
    print("Task b: Testing reasoning capabilities...")
    print("=" * 60)
    results = test_reasoning_capabilities(sections)
    
    print("\n" + "=" * 60)
    print("SUMMARY: LLM Reasoning Capabilities")
    print("=" * 60)
    for r in results:
        print(f"- {r['type']}: Check if answer is correct")

Task a: Loading all sections with tables...
Found table in section: Q1 2024 Financial Highlights (unaudited)
Found table in section: Revenues, Traffic Acquisition Costs (TAC), and Number of Employees
Found table in section: Segment Operating Results
Found table in section: Employee Severance and Related Charges
Found table in section: Contact
Found table in section: (In millions, except par value per share amounts)
Found table in section: Quarter Ended March 31, 2023 2024
Found table in section: Operating activities
Found table in section: Segment Results
Found table in section: Other Income (Expense), Net
Found table in section: Revenues by Geography
Found table in section: Quarter Ended March 31, 2024
Found table in section: Total Revenues — Prior Year Comparative Periods
Found table in section: Quarter Ended March 31,

Found 14 sections with tables

Task b: Testing reasoning capabilities...

[LOOKUP] What was Google's operating margin for Q1 2024?
Expected behavior: 应该返回32%


ReadTimeout: timed out

In [21]:
#Step 0: create summary for tables

def generate_section_summaries(doc):
    """
    generate summary for each table
    """
    sections_info = []
    
    for section in doc.sections():
        html = section.to_html(include_children=True, recurse=True)
        if '<table' not in html.lower():
            continue
        
        text_preview = section.to_text()[:500]  # first 500 chars
        
        sections_info.append({
            'title': section.title,
            'preview': text_preview,
            'html': html,  # save the html
        })
        
        print(f"Section: {section.title}")
        print(f"Preview: {text_preview[:100]}...")
        print("-" * 40)
    
    return sections_info


#Step 1, choose related content

def select_relevant_sections(question: str, sections_info: list, max_sections: int = 2) -> list:
    sections_list = "\n".join([
        f"{i+1}. {s['title']}: {s['preview'][:150]}..." 
        for i, s in enumerate(sections_info)
    ])
    
    prompt = f"""Given the following sections from a financial report, which sections are most relevant to answer this question?

Question: {question}

Available sections:
{sections_list}

Return ONLY the section numbers (e.g., "1, 3") that are most relevant. Return at most {max_sections} sections.
If none are relevant, return "none".

IMPORTANT RULES:
1. If the question asks for a specific metric (like "operating margin", "revenue", "EPS"), 
   look for sections that DIRECTLY contain that exact metric, not sections where you would need to calculate it.
2. "Financial Highlights" sections usually contain summary metrics directly.
3. "Segment Results" sections contain breakdowns by business segment.
4. Choose sections that would give the MOST DIRECT answer.

Answer (just the numbers):"""
    
    resp = llm.complete(prompt)
    response_text = resp.text.strip()
    
    selected_indices = []
    if response_text.lower() != "none":
        import re
        numbers = re.findall(r'\d+', response_text)
        for num in numbers:
            idx = int(num) - 1  # 转换为0-indexed
            if 0 <= idx < len(sections_info):
                selected_indices.append(idx)
    
    # 返回选中的sections
    selected = [sections_info[i] for i in selected_indices[:max_sections]]
    
    print(f"Selected sections for '{question}':")
    for s in selected:
        print(f"  - {s['title']}")
    
    return selected


#Step 2.answer

def query_tables(question: str, selected_sections: list) -> str:
    if not selected_sections:
        return "No relevant sections found for this question."
    
    combined_context = "\n\n".join([
        f"=== {s['title']} ===\n{s['html']}" 
        for s in selected_sections
    ])
    
    prompt = f"""Based on the following tables from a financial report, answer this question:

Question: {question}

Tables:
{combined_context}

Please provide a clear answer. If calculation is needed, show your work step by step.
Answer:"""
    
    resp = llm.complete(prompt)
    return resp.text


# =========test ==========

def test_reasoning_capabilities(sections_info: list):

    test_cases = [
        {
            "question": "What was Google's operating margin for Q1 2024?",
            "type": "lookup",
            "expected": "32%"
        },
        {
            "question": "What is the sum of Google Services revenue and Google Cloud revenue for Q1 2024?",
            "type": "addition",
            "expected": "70,398 + 9,574 = 79,972"
        },
        {
            "question": "What percentage of total revenues comes from Google Cloud in Q1 2024?",
            "type": "percentage",
            "expected": "9,574 / 80,539 ≈ 11.9%"
        },

        {
            "question": "Which segment had the highest operating income in Q1 2024?",
            "type": "comparison",
            "expected": "Google Services ($27,897 million)"
        },
        {
            "question": "How much did total revenues increase from Q1 2023 to Q1 2024?",
            "type": "growth",
            "expected": "80,539 - 69,787 = 10,752 (about 15% growth)"
        },
    ]
    
    results = []
    for test in test_cases:
        print(f"\n{'='*60}")
        print(f"[{test['type'].upper()}] {test['question']}")
        print(f"Expected: {test['expected']}")
        

        selected = select_relevant_sections(test['question'], sections_info)
        

        answer = query_tables(test['question'], selected)
        print(f"\nLLM Answer: {answer}")
        
        results.append({
            'type': test['type'],
            'question': test['question'],
            'answer': answer,
            'expected': test['expected'],
            'sections_used': [s['title'] for s in selected]
        })
    
    return results



if __name__ == "__main__":
    print("=" * 60)
    print("Step 1: Generating section summaries...")
    print("=" * 60)
    sections_info = generate_section_summaries(doc)
    print(f"\nFound {len(sections_info)} sections with tables")
    
    print("\n" + "=" * 60)
    print("Testing reasoning capabilities...")
    print("=" * 60)
    results = test_reasoning_capabilities(sections_info)
    


Step 1: Generating section summaries...
Section: Q1 2024 Financial Highlights (unaudited)
Preview: Q1 2024 Financial Highlights (unaudited)...
----------------------------------------
Section: Revenues, Traffic Acquisition Costs (TAC), and Number of Employees
Preview: Revenues, Traffic Acquisition Costs (TAC), and Number of Employees...
----------------------------------------
Section: Segment Operating Results
Preview: Segment Operating Results...
----------------------------------------
Section: Employee Severance and Related Charges
Preview: Employee Severance and Related Charges...
----------------------------------------
Section: Contact
Preview: Contact...
----------------------------------------
Section: (In millions, except par value per share amounts)
Preview: (In millions, except par value per share amounts)...
----------------------------------------
Section: Quarter Ended March 31, 2023 2024
Preview: Quarter Ended March 31, 2023 2024...
-------------------------------------